In [1]:
import kerastuner as kt
import tensorflow as tf
import tensorflow_datasets as tfds

def build_model(hp):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs
    
    for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
        filters = hp.Int('filters_' + str(i), 32, 256, step=32)
        
        for _ in range(2):
            x = tf.keras.layers.Convolution2D(filters, kernel_size=(3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)
        
        if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
            x = tf.keras.layers.MaxPool2D()(x)
        
        else:
            x = tf.keras.layers.AvgPool2D()(x)
            
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense(hp.Int('hidden_size', 30, 100, step=10, default=50), activation='relu')(x)
    x = tf.keras.layers.Dropout(hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(
        hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2)

data = tfds.load('cifar10')
train_ds, test_ds = data['train'], data['test']


def standardize_record(record):
    return tf.cast(record['image'], tf.float32) / 255., record['label']


train_ds = train_ds.map(standardize_record).cache().batch(64).shuffle(10000)
test_ds = test_ds.map(standardize_record).cache().batch(64)

tuner.search(train_ds,
             validation_data=test_ds,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=0)])

best_model = tuner.get_best_models(1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
Epoch 1/2


    166/Unknown - 3s 3s/step - loss: 2.4474 - accuracy: 0.07 - 4s 2s/step - loss: 2.4367 - accuracy: 0.09 - 5s 2s/step - loss: 2.3744 - accuracy: 0.09 - 6s 1s/step - loss: 2.3858 - accuracy: 0.10 - 7s 1s/step - loss: 2.3856 - accuracy: 0.09 - 8s 1s/step - loss: 2.3650 - accuracy: 0.09 - 9s 1s/step - loss: 2.3763 - accuracy: 0.09 - 9s 1s/step - loss: 2.3683 - accuracy: 0.09 - 10s 1s/step - loss: 2.3611 - accuracy: 0.097 - 11s 1s/step - loss: 2.3606 - accuracy: 0.093 - 12s 1s/step - loss: 2.3556 - accuracy: 0.098 - 13s 1s/step - loss: 2.3496 - accuracy: 0.106 - 14s 1s/step - loss: 2.3469 - accuracy: 0.110 - 15s 1s/step - loss: 2.3355 - accuracy: 0.116 - 15s 1s/step - loss: 2.3293 - accuracy: 0.120 - 16s 1s/step - loss: 2.3254 - accuracy: 0.123 - 17s 1s/step - loss: 2.3242 - accuracy: 0.125 - 18s 1s/step - loss: 2.3185 - accuracy: 0.131 - 19s 993ms/step - loss: 2.3157 - accuracy: 0.12 - 20s 985ms/step - loss: 2.3135 - accuracy: 0.12 - 21s 978ms/step - loss: 2.3078 - accuracy: 0.12 - 21s 9

    327/Unknown - 145s 870ms/step - loss: 1.9892 - accuracy: 0.268 - 146s 870ms/step - loss: 1.9875 - accuracy: 0.269 - 147s 870ms/step - loss: 1.9867 - accuracy: 0.269 - 148s 870ms/step - loss: 1.9851 - accuracy: 0.270 - 149s 870ms/step - loss: 1.9835 - accuracy: 0.270 - 150s 870ms/step - loss: 1.9824 - accuracy: 0.271 - 150s 870ms/step - loss: 1.9812 - accuracy: 0.271 - 151s 870ms/step - loss: 1.9794 - accuracy: 0.271 - 152s 870ms/step - loss: 1.9786 - accuracy: 0.271 - 153s 869ms/step - loss: 1.9772 - accuracy: 0.271 - 154s 869ms/step - loss: 1.9759 - accuracy: 0.272 - 155s 869ms/step - loss: 1.9752 - accuracy: 0.272 - 156s 869ms/step - loss: 1.9737 - accuracy: 0.273 - 156s 869ms/step - loss: 1.9720 - accuracy: 0.273 - 157s 869ms/step - loss: 1.9715 - accuracy: 0.273 - 158s 869ms/step - loss: 1.9698 - accuracy: 0.274 - 159s 869ms/step - loss: 1.9692 - accuracy: 0.274 - 160s 869ms/step - loss: 1.9685 - accuracy: 0.275 - 161s 869ms/step - loss: 1.9675 - accuracy: 0.275 - 162s 868ms/st

    488/Unknown - 283s 862ms/step - loss: 1.7994 - accuracy: 0.341 - 284s 862ms/step - loss: 1.7983 - accuracy: 0.341 - 285s 862ms/step - loss: 1.7975 - accuracy: 0.341 - 285s 862ms/step - loss: 1.7965 - accuracy: 0.342 - 286s 862ms/step - loss: 1.7959 - accuracy: 0.342 - 287s 862ms/step - loss: 1.7950 - accuracy: 0.342 - 288s 862ms/step - loss: 1.7940 - accuracy: 0.343 - 289s 862ms/step - loss: 1.7929 - accuracy: 0.343 - 290s 862ms/step - loss: 1.7915 - accuracy: 0.344 - 291s 862ms/step - loss: 1.7903 - accuracy: 0.345 - 291s 862ms/step - loss: 1.7898 - accuracy: 0.345 - 292s 862ms/step - loss: 1.7894 - accuracy: 0.345 - 293s 862ms/step - loss: 1.7883 - accuracy: 0.345 - 294s 862ms/step - loss: 1.7878 - accuracy: 0.346 - 295s 862ms/step - loss: 1.7873 - accuracy: 0.346 - 296s 862ms/step - loss: 1.7867 - accuracy: 0.347 - 297s 862ms/step - loss: 1.7862 - accuracy: 0.347 - 297s 862ms/step - loss: 1.7854 - accuracy: 0.347 - 298s 862ms/step - loss: 1.7844 - accuracy: 0.348 - 299s 862ms/st

    649/Unknown - 420s 858ms/step - loss: 1.6690 - accuracy: 0.397 - 421s 858ms/step - loss: 1.6677 - accuracy: 0.397 - 421s 858ms/step - loss: 1.6664 - accuracy: 0.398 - 422s 858ms/step - loss: 1.6658 - accuracy: 0.398 - 423s 858ms/step - loss: 1.6646 - accuracy: 0.399 - 424s 858ms/step - loss: 1.6639 - accuracy: 0.399 - 425s 858ms/step - loss: 1.6631 - accuracy: 0.399 - 426s 858ms/step - loss: 1.6622 - accuracy: 0.400 - 426s 858ms/step - loss: 1.6617 - accuracy: 0.400 - 427s 858ms/step - loss: 1.6608 - accuracy: 0.400 - 428s 858ms/step - loss: 1.6601 - accuracy: 0.400 - 429s 858ms/step - loss: 1.6596 - accuracy: 0.401 - 430s 858ms/step - loss: 1.6590 - accuracy: 0.401 - 431s 858ms/step - loss: 1.6579 - accuracy: 0.402 - 432s 858ms/step - loss: 1.6573 - accuracy: 0.402 - 432s 858ms/step - loss: 1.6564 - accuracy: 0.402 - 433s 858ms/step - loss: 1.6560 - accuracy: 0.403 - 434s 858ms/step - loss: 1.6549 - accuracy: 0.403 - 435s 858ms/step - loss: 1.6541 - accuracy: 0.403 - 436s 858ms/st

...

    493/Unknown - 119s 357ms/step - loss: 1.7329 - accuracy: 0.355 - 119s 357ms/step - loss: 1.7315 - accuracy: 0.355 - 119s 357ms/step - loss: 1.7305 - accuracy: 0.356 - 120s 357ms/step - loss: 1.7295 - accuracy: 0.356 - 120s 357ms/step - loss: 1.7289 - accuracy: 0.356 - 121s 357ms/step - loss: 1.7282 - accuracy: 0.356 - 121s 357ms/step - loss: 1.7278 - accuracy: 0.357 - 121s 357ms/step - loss: 1.7272 - accuracy: 0.357 - 122s 357ms/step - loss: 1.7267 - accuracy: 0.357 - 122s 357ms/step - loss: 1.7257 - accuracy: 0.357 - 122s 357ms/step - loss: 1.7255 - accuracy: 0.357 - 123s 357ms/step - loss: 1.7247 - accuracy: 0.358 - 123s 357ms/step - loss: 1.7238 - accuracy: 0.358 - 123s 357ms/step - loss: 1.7231 - accuracy: 0.358 - 124s 357ms/step - loss: 1.7226 - accuracy: 0.359 - 124s 357ms/step - loss: 1.7218 - accuracy: 0.359 - 124s 357ms/step - loss: 1.7202 - accuracy: 0.359 - 125s 357ms/step - loss: 1.7197 - accuracy: 0.359 - 125s 357ms/step - loss: 1.7196 - accuracy: 0.360 - 126s 357ms/st

    654/Unknown - 175s 354ms/step - loss: 1.6178 - accuracy: 0.401 - 175s 354ms/step - loss: 1.6173 - accuracy: 0.401 - 176s 354ms/step - loss: 1.6164 - accuracy: 0.401 - 176s 354ms/step - loss: 1.6163 - accuracy: 0.401 - 176s 354ms/step - loss: 1.6157 - accuracy: 0.402 - 177s 354ms/step - loss: 1.6146 - accuracy: 0.402 - 177s 354ms/step - loss: 1.6139 - accuracy: 0.403 - 177s 354ms/step - loss: 1.6140 - accuracy: 0.403 - 178s 354ms/step - loss: 1.6134 - accuracy: 0.403 - 178s 354ms/step - loss: 1.6125 - accuracy: 0.403 - 178s 354ms/step - loss: 1.6119 - accuracy: 0.403 - 179s 354ms/step - loss: 1.6114 - accuracy: 0.404 - 179s 354ms/step - loss: 1.6111 - accuracy: 0.404 - 179s 354ms/step - loss: 1.6110 - accuracy: 0.404 - 180s 354ms/step - loss: 1.6100 - accuracy: 0.404 - 180s 354ms/step - loss: 1.6095 - accuracy: 0.404 - 181s 354ms/step - loss: 1.6087 - accuracy: 0.405 - 181s 354ms/step - loss: 1.6088 - accuracy: 0.405 - 181s 354ms/step - loss: 1.6079 - accuracy: 0.405 - 182s 354ms/st

    773/Unknown - 231s 353ms/step - loss: 1.5377 - accuracy: 0.435 - 231s 353ms/step - loss: 1.5370 - accuracy: 0.435 - 232s 353ms/step - loss: 1.5368 - accuracy: 0.435 - 232s 353ms/step - loss: 1.5365 - accuracy: 0.436 - 233s 353ms/step - loss: 1.5360 - accuracy: 0.436 - 233s 353ms/step - loss: 1.5356 - accuracy: 0.436 - 233s 353ms/step - loss: 1.5351 - accuracy: 0.436 - 234s 353ms/step - loss: 1.5345 - accuracy: 0.436 - 234s 353ms/step - loss: 1.5339 - accuracy: 0.437 - 234s 353ms/step - loss: 1.5334 - accuracy: 0.437 - 235s 353ms/step - loss: 1.5332 - accuracy: 0.437 - 235s 353ms/step - loss: 1.5327 - accuracy: 0.437 - 235s 353ms/step - loss: 1.5322 - accuracy: 0.437 - 236s 353ms/step - loss: 1.5319 - accuracy: 0.438 - 236s 353ms/step - loss: 1.5314 - accuracy: 0.438 - 236s 353ms/step - loss: 1.5310 - accuracy: 0.438 - 237s 353ms/step - loss: 1.5303 - accuracy: 0.438 - 237s 353ms/step - loss: 1.5299 - accuracy: 0.438 - 237s 353ms/step - loss: 1.5290 - accuracy: 0.439 - 238s 353ms/st

    773/Unknown - 272s 352ms/step - loss: 1.4861 - accuracy: 0.4563

KeyboardInterrupt: 